In [1]:
from neo4j import GraphDatabase

WINDOWS_IP = "28.11.5.39"
URI = f"bolt://{WINDOWS_IP}:7687"
USERNAME = "neo4j"
PASSWORD = "phongthang2012"

driver = GraphDatabase.driver(URI, auth=(USERNAME, PASSWORD))


def test_connection():
    with driver.session() as session:
        result = session.run("RETURN 'Connected to Remote Neo4j!' AS message")
        for record in result:
            print(record["message"])

test_connection()

Connected to Remote Neo4j!


In [2]:
# from neo4j_graphrag.retrievers import VectorRetriever
# from neo4j_graphrag.llm import OpenAILLM
# from neo4j_graphrag.generation import GraphRAG
from neo4j_graphrag.embeddings.sentence_transformers import SentenceTransformerEmbeddings
# INDEX_NAME = "vector_index_name"
embedder = SentenceTransformerEmbeddings(model="dangvantuan/vietnamese-document-embedding", trust_remote_code=True)

# # Initialize the retriever
# retriever = VectorRetriever(driver, INDEX_NAME, embedder)

# # 3. LLM
# # Note: the OPENAI_API_KEY must be in the env vars
# llm = OpenAILLM(model_name="gpt-4o", model_params={"temperature": 0})

# # Initialize the RAG pipeline
# rag = GraphRAG(retriever=retriever, llm=llm)

# # Query the graph
# query_text = "Điều 13"
# response = rag.search(query_text=query_text, retriever_config={"top_k": 5})
# print(response.answer)


/home/phongnd/anaconda3/envs/llm2/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# EMBEDDING_MODEL = "dangvantuan/vietnamese-document-embedding"
# QWEN_MODEL = "AITeamVN/Vi-Qwen2-3B-RAG"
# from sentence_transformers import SentenceTransformer
# from transformers import AutoModelForCausalLM, AutoTokenizer
# import faiss
# import torch
# import numpy as np
# import json

# # === CONNECT TO NEO4J ===
# class Neo4jHandler:
#     def __init__(self, uri, user, password):
#         self.driver = GraphDatabase.driver(uri, auth=(user, password))

#     def close(self):
#         self.driver.close()

#     def get_nodes_with_content(self):
#         query = """
#         MATCH (n) WHERE n.content IS NOT NULL
#         RETURN n.id AS id, n.content AS content, labels(n) AS type
#         """
#         with self.driver.session() as session:
#             result = session.run(query)
#             return [record for record in result]

#     def get_graph_context(self, node_ids):
#         """ Retrieve related legal clauses via Neo4j Graph Search """
#         query = f"""
#         MATCH (n) WHERE n.id IN {json.dumps(node_ids)}
#         OPTIONAL MATCH (n)-[:PARENT*]->(p)   // Get parent context
#         OPTIONAL MATCH (n)-[:NEXT]->(next)   // Get sequential context
#         RETURN n.id, n.content, p.content AS parent_content, next.content AS next_content
#         """
#         with self.driver.session() as session:
#             results = session.run(query)
#             return [record for record in results]

# # === EMBEDDING & VECTOR STORAGE ===
# class VectorStore:
#     def __init__(self, model_name):
#         self.model = SentenceTransformer(model_name, trust_remote_code=True)
#         self.index = None
#         self.id_map = {}  # FAISS ID -> Document ID

#     def build_index(self, docs):
#         embeddings = np.array([self.model.encode(d["content"]) for d in docs], dtype="float32")
#         self.index = faiss.IndexFlatL2(embeddings.shape[1])
#         self.index.add(embeddings)
#         for i, d in enumerate(docs):
#             self.id_map[i] = d["id"]

#     def search(self, query, top_k=5):
#         query_vec = np.array([self.model.encode(query)], dtype="float32")
#         _, indices = self.index.search(query_vec, top_k)
#         return [self.id_map[i] for i in indices[0] if i in self.id_map]

# # === LLM GENERATION (QWEN) ===
# class QwenRAG:
#     def __init__(self, model_name):
#         self.tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
#         self.model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", torch_dtype=torch.float16, trust_remote_code=True)

#     def generate(self, query, context):
#         prompt = f"Bạn là một trợ lí Tiếng Việt nhiệt tình và trung thực. Hãy luôn trả lời một cách hữu ích nhất có thể.\n\n"
        
#         for i, doc in enumerate(context):
#             prompt += f"[{i+1}] {doc['content']}\n"
#             if doc["parent_content"]:
#                 prompt += f"-> Điều khoản liên quan: {doc['parent_content']}\n"
#             if doc["next_content"]:
#                 prompt += f"-> Điều khoản tiếp theo: {doc['next_content']}\n"
#         prompt += f"\nHỏi: {query}\nĐáp:"

#         input_ids = self.tokenizer(prompt, return_tensors="pt").input_ids.to("cuda")
#         output_ids = self.model.generate(input_ids, max_length=1024)
#         return self.tokenizer.decode(output_ids[0], skip_special_tokens=True)

# # === MAIN SCRIPT ===
# if __name__ == "__main__":
#     neo4j_handler = Neo4jHandler(URI, USERNAME, PASSWORD)

#     # Step 1: Extract nodes
#     nodes = neo4j_handler.get_nodes_with_content()

#     # Step 2: Build FAISS vector index
#     vector_store = VectorStore(EMBEDDING_MODEL)
#     vector_store.build_index(nodes)

#     # Step 3: Query the legal database
#     query_text = "Điều kiện để phân phối thuốc"  # Example query
#     result_ids = vector_store.search(query_text)

#     # Step 4: Retrieve Graph Context from Neo4j
#     if result_ids:
#         graph_context = neo4j_handler.get_graph_context(result_ids)
        
#         # Step 5: Generate Answer using Qwen
#         qwen_rag = QwenRAG(QWEN_MODEL)
#         response = qwen_rag.generate(query_text, graph_context)

#         print("==== Câu trả lời của AI ====")
#         print(response)

#     neo4j_handler.close()

In [4]:
from langchain_community.vectorstores import Neo4jVector
# from langchain_openai import OpenAIEmbeddings
from langchain.embeddings import HuggingFaceEmbeddings
import os 
from dotenv import load_dotenv
embedding_model = HuggingFaceEmbeddings(
    model_name="dangvantuan/vietnamese-document-embedding", 
    model_kwargs={"trust_remote_code": True},)
load_dotenv()

# Create the vectorstore for our existing graph
paper_graph = Neo4jVector.from_existing_graph(
    embedding=embedding_model,
    url="bolt://28.11.5.39:7687",
    username="neo4j",
    password="phongthang2012",
    index_name="doc_index",
    node_label="C_Node",
    text_node_properties=["content"],
    embedding_node_property="doc_embedding",
)

/tmp/ipykernel_6168/184582308.py:6: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(


In [10]:
from pprint import pprint

result = paper_graph.similarity_search("Cơ sở phân phối")
for i in result:
    pprint(i.page_content)

('\n'
 'content: 4. Cơ sở phân phối là cơ sở thực hiện hoạt động phân phối thuốc, '
 'nguyên liệu làm thuốc, cơ sở bán buôn thuốc, nguyên liệu làm thuốc, cơ sở '
 'phân phối vắc xin trong Chương trình Tiêm chủng mở rộng quốc gia ở tuyến '
 'tỉnh, tuyến huyện và cơ sở khác có hoạt động phân phối thuốc, nguyên liệu '
 'làm thuốc không vì mục đích thương mại.')
('\n'
 'content: 2. Phân phối nguyên liệu làm thuốc là việc phân chia và di chuyển, '
 'bảo quản nguyên liệu làm thuốc trong quá trình di chuyển từ kho của cơ sở '
 'sản xuất nguyên liệu làm thuốc, cơ sở nhập khẩu nguyên liệu làm thuốc đến cơ '
 'sở sản xuất thuốc thành phẩm hoặc đến các điểm phân phối, bảo quản của cơ sở '
 'phân phối hoặc giữa các điểm phân phối bằng các phương tiện vận chuyển khác '
 'nhau.')
('\n'
 'content: 1. Phân phối thuốc là hoạt động phân chia và di chuyển, bảo quản '
 'thuốc trong quá trình di chuyển từ kho của cơ sở sản xuất thuốc, nhập khẩu '
 'thuốc hoặc từ cơ sở phân phối cho đến người sử dụng hoặc đ

In [5]:
driver.close()